In [36]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import tqdm
import torch
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
#%matplotlib inline

In [37]:
with open("res.txt","r") as f:
    annos = f.readlines()
annos = [np.array([float(x) for x in anno.split()]) for anno in annos]
annos = np.stack(annos)

In [38]:
pattern = "generate_with_latent/seed{}.npy"
xs = []
for idx in range(1,len(annos)+1):
    x = np.load(pattern.format(str(idx).zfill(4)))
    xs.append(x[:,4])
xs = np.concatenate(xs).reshape(len(annos),-1)


In [39]:
clf = LDA()
clf.fit(xs[:800], annos[:800,1].astype(int))
pattern_vector = clf.coef_[:,:512]
pattern_vector /= np.linalg.norm(pattern_vector,ord=2)
torch.tensor(pattern_vector)
torch.save(torch.tensor(pattern_vector),"pattern_strength.pt")

In [40]:
sleeve_vector = np.load("eigen.npy")
print(np.real(sleeve_vector).__pow__(2).sum())
print(np.imag(sleeve_vector).__pow__(2).sum())
sleeve_vector = np.real(sleeve_vector[:512]).reshape(1,512)
sleeve_vector /= np.linalg.norm(sleeve_vector,ord=2)
#torch.save(torch.tensor(sleeve_vector), "sleeve_length.pt")

234
16
159
52
83
49
38
12
33
6
318
512
0.3340591807026586
0.25094382304405316


In [41]:

preds = []
for x in xs:
    preds.append(sleeve_vector[0]@x[:512])
preds = np.array(preds)
print(np.corrcoef(preds, annos[:,0]))
ref = -torch.load("upper_length.pt",map_location=torch.device('cpu')).numpy()
ref = ref.reshape(1,512)/np.linalg.norm(ref,ord=2)
preds = []
for x in xs:
    preds.append(ref[0]@x[:512])
preds = np.array(preds)
print(np.corrcoef(preds, annos[:,0]))
print(np.linalg.norm(sleeve_vector-ref,ord=2))
print(np.linalg.norm(sleeve_vector-pattern_vector,ord=2))
print(np.linalg.norm(ref-pattern_vector,ord=2))

[[ 1.         -0.39112808]
 [-0.39112808  1.        ]]
[[1.         0.57878992]
 [0.57878992 1.        ]]
1.4478500713218505
1.4572130305821922
1.3677601
